# Лабораторная работа 4. Алгоритмы для решения Quadratic Assignment Problem

## Мельников Евгений 18ПМИ-1. Малышева Екатерина 18ПМИ-2.


In [1]:
import time
from google.colab import drive
import os
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
from math import gcd, floor
from random import randint
import time
from itertools import *
from scipy.optimize import linprog
import copy
import random
from tqdm import tqdm


In [3]:
drive.mount('/drive')
os.listdir('/drive/MyDrive/qap_benchmarks')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


['tai100a.txt',
 'tai80a.txt',
 'tai20a.txt',
 'tai40a.txt',
 'tai60a.txt',
 'output']

## 2-opt local search with first improvement + don't look bits


In [4]:
class qap:

  def __str__(self):
      return 'qap'
      

  def __init__(self, distances, flows, n):
    self.n = n
    self.distances = distances
    self.flows = flows 
    self.best_solution = list(range(0, n)) # initial solution
    random.shuffle(self.best_solution)
    self.best_func = self.calc_opt_f(self.best_solution)
    
  def calc_opt_f(self, positions): #[2, 3, 5, 6] 
    func = 0
    for i in range(self.n): 
      for j in range(self.n): # we iterate over CITIES
              
        #func += self.flows[i][j] * self.distances[positions[i]][positions[j]]

        # positions[i] returns the index of the PLANT located in the i-th city
        # we multiply the distances between city i and city j by the flow between the plants located in those cities
        # In order to get the index of the plant, located in the i-th or j-th city, we use the positions list
        # Thus, this formula should be correct
        func += self.flows[positions[i]][positions[j]] * self.distances[i][j] 
          
          
    return func

  
  def swap(self, lst, ind1, ind2): 
    tmp = lst[ind1]
    lst[ind1] = lst[ind2]
    lst[ind2] = tmp
    return lst


  def local_search(self): 
    swap_count = np.zeros(self.n, dtype=int)  # stores numbers of permutations without improvements for every plant
    dlb = np.zeros(self.n, dtype=int)  # don't look bits. If the number of permutations of a plant is equal to n-1, then we add it to dlb ...
    # ... and don't swap it anymore
    start_time = time.time()

    def local_search_rec(current_solution):
      #print(dlb)
      for i in range(self.n-1): # iterate over Cities
        for j in range(i+1, self.n):
          if dlb[current_solution[j]] == True or dlb[current_solution[i]] == True: # we don't swap the plants if one of them is in dlb list
            continue
          else:
            self.swap(current_solution, i, j)
            current_func = self.calc_opt_f(current_solution)
            if current_func < self.best_func:
              self.best_solution = current_solution
              self.best_func = current_func

              return local_search_rec(current_solution) # first improvement?

            else:
              swap_count[current_solution[i]] += 1 # if swapping plants hasn't improved the function
              swap_count[current_solution[j]] += 1
              if swap_count[current_solution[i]] == self.n - 1: # if a plant has been swapped n - 1 times, then we add it to dlb
                dlb[current_solution[i]] = True
              if swap_count[current_solution[j]] == self.n - 1:
                dlb[current_solution[j]] = True

    local_search_rec(self.best_solution)


    return self.best_solution, self.best_func, time.time() - start_time



## Iterated local search + stochastic 2-opt


In [5]:
class qap_iterated(qap):

    def __str__(self):
      return 'qap_iterated'

    def __init__(self, distances, flows, n, solution = False):
        self.n = n
        self.distances = distances
        self.flows = flows 
        if solution:
            self.best_solution = solution
        else:
            self.best_solution = list(np.random.permutation(n))
        self.best_func = self.calc_opt_f(self.best_solution)
    
    def __name__(self):
      return 'qap_iterated'
    
    def stochastic_local_search(self, permut_max, solution = False):
        
        if not solution:
            solution = self.best_solution
            
        tmp_best_sol = []
        curr_sol=[]
        for el in solution:
            tmp_best_sol.append(el)
            curr_sol.append(el)    
            
        tmp_best_f = self.calc_opt_f(tmp_best_sol)
        curr_f = self.calc_opt_f(curr_sol) 
        
        
        for i in range(permut_max):
            
            ind_l = random.randint(0, self.n - 2)
            ind_r = random.randint(ind_l + 1, self.n-1)
            
            curr_sol[ind_l:ind_r] = curr_sol[ind_l:ind_r][::-1]
            curr_f = self.calc_opt_f(curr_sol)
            
            if curr_f < tmp_best_f:
                tmp_best_sol = curr_sol
                tmp_best_f = curr_f
                
        return tmp_best_sol, tmp_best_f
    
    def iterated_local_search(self, iters):
        start = time.time()
        
        self.best_solution, self.best_func = self.stochastic_local_search(iters//5)
        
        for i in range(iters):
            #perturbation
            sol, f = self.best_solution, self.best_func
            
            k = random.randint(1, self.n-1)
            inds = np.random.choice(self.n, k, replace=False)
            shuffled_inds = np.random.permutation(inds)
            new_sol = sol.copy()
            for i in range(k):
                new_sol[inds[i]] = sol[shuffled_inds[i]]
            
            #local search (stochastic)
            new_sol_opt, new_sol_opt_f = self.stochastic_local_search(iters//5, new_sol)
            
            #acceptance criterion
            if new_sol_opt_f < f:
                self.best_solution, self.best_func = new_sol_opt, new_sol_opt_f 
        
        end = time.time() - start
        return self.best_solution, self.best_func, end

# Benchmarks


In [6]:
class benchmarks_qap:
  # 
  def __init__(self, path, verbose=False): # path to a folder, which contains benchmark files
    self.benchmark_names = []
    self.benchmark_n = {}
    self.benchmark_distances = {}
    self.benchmark_flows = {}
    
    
    self.verbose = verbose
    self.path = path
    if self.verbose:
      print("Opening up qap files...")

    for filename in os.listdir(path):
      if filename != "output":
        benchmark_name = filename[:-4]
        print(benchmark_name)
        self.benchmark_names.append(benchmark_name)      
        text = open(path + '/' + filename)

        n = int(text.readline())
        self.benchmark_n[benchmark_name] = n
        
        self.benchmark_distances[benchmark_name] = []
        for i in range(n):
          self.benchmark_distances[benchmark_name].append(list(map(int, text.readline().split())))

        text.readline()

        self.benchmark_flows[benchmark_name] = []
        for i in range(n):
          self.benchmark_flows[benchmark_name].append(list(map(int, text.readline().split())))

    if self.verbose:
      print("Finished")      


  def check_single_benchmark(self, algorithm, times_repeat=5): 
    mean_time = 0    
    best_found_sol =[]
    best_found_f = 9223372036854775807
    #print(algorithm)
    for i in range(times_repeat):
      if self.verbose:
        print("Try", algorithm,  i)
      if algorithm.__str__()  == 'qap': 
        solution, func, time = algorithm.local_search()
      else:
        solution, func, time = algorithm.iterated_local_search(100)
      if func < best_found_f:
        best_found_f = func
        best_found_sol = solution

      mean_time += time

    return best_found_sol, best_found_f, mean_time/times_repeat


  def report(self):
    time_table = {}
    func_table = {}
    #sol_table = {}

    for benchmark_name in self.benchmark_names:
      if self.verbose:
        print("Running %s..." %(benchmark_name))
      
      file = open(self.path + "/output/" + benchmark_name + ".sol", "w")

      time_table[benchmark_name] = [] 
      func_table[benchmark_name] = []
      #sol_table[benchmark_name] = []

      results = self.check_single_benchmark(
          qap(self.benchmark_distances[benchmark_name], self.benchmark_flows[benchmark_name], self.benchmark_n[benchmark_name])
      )
      
      
      #sol_table[benchmark_name].append(results[0])
      time_table[benchmark_name].append(results[2])
      func_table[benchmark_name].append(results[1])
      file.write(" ".join(str(x) for x in results[0]))
      file.write("\n")

      results = self.check_single_benchmark(
          qap_iterated(self.benchmark_distances[benchmark_name], self.benchmark_flows[benchmark_name], self.benchmark_n[benchmark_name])
      )
      #sol_table[benchmark_name].append(results[0])
      time_table[benchmark_name].append(results[2]) 
      func_table[benchmark_name].append(results[1])
      file.write(" ".join(str(x) for x in results[0]))
      
      file.close()


    time_table = pd.DataFrame.from_dict(time_table)
    func_table = pd.DataFrame.from_dict(func_table)
    #sol_table = pd.DataFrame.from_dict(sol_table)

    time_table = time_table.rename(index={0: '2-opt local search with first improvement + don\'t look bits',
                                          1: 'Iterated local search + stochastic 2-opt, iter = 100'})
    func_table = func_table.rename(index={0: '2-opt local search with first improvement + don\'t look bits',
                                          1: 'Iterated local search + stochastic 2-opt, iter = 100'})
    #sol_table = sol_table.rename(index={0: '2-opt local search with first improvement + don\'t look bits',
    #                                      1: 'Iterated local search + stochastic 2-opt, iter = 100'})
    


    return time_table, func_table


In [7]:
benchmarks = benchmarks_qap('/drive/MyDrive/qap_benchmarks', verbose=False)

time_table, func_table = benchmarks.report()

tai100a
tai80a
tai20a
tai40a
tai60a


# Время выполнения алгоритмов:

In [8]:
time_table

,tai100a,tai80a,tai20a,tai40a,tai60a
2-opt local search with first improvement + don't look bits,10.258241,4.696206,0.018215,0.301681,1.438983
"Iterated local search + stochastic 2-opt, iter = 100",6.228793,4.019396,0.269897,1.031936,2.294664


# Лучший результат работы алгоритмов:

In [9]:
func_table

,tai100a,tai80a,tai20a,tai40a,tai60a
2-opt local search with first improvement + don't look bits,23649456,15296932,831480,3594708,8284414
"Iterated local search + stochastic 2-opt, iter = 100",23621576,15258976,784994,3603496,8228784
